# Data Acquisition

In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import os, re, csv
from acquire import *

## Codeup Blog Articles

1. Scrape the article text from the following pages:

> - https://codeup.com/codeups-data-science-career-accelerator-is-here/
> - https://codeup.com/data-science-myths/
> - https://codeup.com/data-science-vs-data-analytics-whats-the-difference/
> - https://codeup.com/10-tips-to-crush-it-at-the-sa-tech-job-fair/
> - https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/

Encapsulate your work in a function named `get_blog_articles` that will return a list of dictionaries, with each dictionary representing one article. The shape of each dictionary should look like this:

``` python
{
    'title': 'the title of the article',
    'content': 'the full text content of the article'
}
```

In [2]:
urls = [
    'https://codeup.com/codeups-data-science-career-accelerator-is-here/',
    'https://codeup.com/data-science-myths/',
    'https://codeup.com/data-science-vs-data-analytics-whats-the-difference/',
    'https://codeup.com/10-tips-to-crush-it-at-the-sa-tech-job-fair/',
    'https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/'
]
headers = {
    'User-Agent': 'Codeup Data Science'
}
        
articles = list()
for url in urls:
    articles.append(get_blog_article(url))


In [3]:
target = 'https://codeup.com/blog/'
df = fetch_all_urls(target)


Parsed https://codeup.com/help/linkedin?trk=d_checkpoint_lg_consumerLogin_ft_send_feedback&amp;lang=en
Invalid URL '/checkpoint/rp/request-password-reset?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2FshareArticle%3Furl%3Dhttps%3A%2F%2Fwww%2Ecoursereport%2Ecom%2Fschools%2Fcodeup%2523%252Foutcomes': No schema supplied. Perhaps you meant http:///checkpoint/rp/request-password-reset?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2FshareArticle%3Furl%3Dhttps%3A%2F%2Fwww%2Ecoursereport%2Ecom%2Fschools%2Fcodeup%2523%252Foutcomes?
Parsed /checkpoint/rp/request-password-reset?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2FshareArticle%3Furl%3Dhttps%3A%2F%2Fwww%2Ecoursereport%2Ecom%2Fschools%2Fcodeup%2523%252Foutcomes

Parsed https://codeup.com/legal/cookie-policy?trk=d_checkpoint_lg_consumerLogin_ft_cookie_policy

Parsed https://codeup.com/legal/copyright-policy?trk=d_checkpoint_lg_consumerLogin_ft_copyright_policy


/usr/local/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Parsed https://codeup.com/wp-login.php?action=lostpassword

Parsed https://codeup.com/about?src=coursereport

Parsed https://codeup.com/images/disclosures/state_consumer_rights_nm.pdf

Parsed https://codeup.com/licenses
Parsed https://codeup.com/privacy

Parsed https://codeup.com/student-portal?src=coursereport

Parsed https://codeup.com/images/disclosures/state_consumer_rights_sc.pdf

Parsed https://codeup.com/compare/vetting?src=coursereport

Parsed https://codeup.com/images/disclosures/state_fee_disclosure_nm.pdf

Parsed https://codeup.com/cdn-cgi/l/email-protection#472528283324262a372f222b3707263424222933213229232e29206924282a
Parsed https://codeup.com/#payment_calc_preview_scroll

Parsed https://codeup.com/cdn-cgi/l/email-protection

Parsed https://codeup.com/tel:8772792614

Parsed https://codeup.com/cdn-cgi/l/email-protection#60020f0f1403010d1008050c1020011303050e1406150e04090e074e030f0d

Parsed https://codeup.com/compare/vetting

Parsed https://codeup.com/student-portal
Invalid 

Parsed https://www.tpr.org/technology-entrepreneurship/2021-04-16/codeup-buys-rackspace-cloud-academy

Parsed https://codeup.com/wd-application/
Parsed https://therivardreport.com/codeup-set-to-open-coding-school-in-dallas/

Parsed https://codeup.com/codeup-dallas-2020/
Parsed https://codeup.com/dallas-3
Parsed https://codeup.com/alumni-stories/my-codeup-experience-mario-m/
Parsed https://codeup.com/alumni-stories/student-blog-sharon-r-a-leap-of-logic/
Parsed https://codeup.com/alumni-stories/scholarship-recipient-kris-campos/
Parsed https://codeup.com/alumni-stories/my-experience-at-codeup-mario-m-2/
Parsed https://codeup.com/alumni-stories/student-profile-leslie-tolbert/
HTTPSConnectionPool(host='crowdfund.codeup20.wpengine.com', port=443): Max retries exceeded with url: /campaigns/3 (Caused by SSLError(SSLCertVerificationError("hostname 'crowdfund.codeup20.wpengine.com' doesn't match either of '*.staging.wpengine.com', 'staging.wpengine.com'")))
Parsed https://crowdfund.codeup20.wpe

string index out of range
Parsed https://www.eventbrite.com/e/codeup-learn-to-code-html-css-intro-to-web-development-tickets-151240397099?aff=ebdsoporgprofile
string index out of range
Parsed https://www.eventbrite.com/e/codeup-virtual-data-scientist-day-easley-class-tickets-155688216647?aff=ebdsoporgprofile
string index out of range
Parsed https://www.eventbrite.com/e/codeup-learn-to-code-workshop-python-intro-to-data-science-tickets-151120446323?aff=ebdsoporgprofile
string index out of range
Parsed https://www.eventbrite.com/e/codeup-virtual-info-session-with-our-ceo-tickets-196180630547?aff=ebdsoporgprofile
string index out of range
Parsed https://www.eventbrite.com/e/codeup-what-is-data-science-tickets-155701414121?aff=ebdsoporgprofile
Parsed https://www.facebook.com/gocodeup
string index out of range
Parsed https://www.eventbrite.com/e/codeup-learn-to-code-workshop-javascript-intro-to-web-development-tickets-160031389193?aff=ebdsoporgprofile
string index out of range
Parsed https:

...


Parsed https://codeup.com/uploads/241-1473171326.pdf

Parsed https://codeup.com/mailto:Dklunariley@gmail.com

Parsed https://codeup.com/uploads/189-1475558975.pdf

Parsed https://codeup.com/uploads/195-1469197814.pdf

Parsed https://codeup.com/mailto:will78006@yahoo.com

Parsed https://codeup.com/uploads/204-1465944360.pdf

Parsed https://codeup.com/mailto:alaurit@gmail.com

Parsed https://codeup.com/mailto:michael.d.truong@icloud.com

Parsed https://codeup.com/mailto:a_s_crawford@hotmail.com

Parsed https://codeup.com/mailto:chris@chrisdavila.net

Parsed https://codeup.com/uploads/239-1474603124.pdf

Parsed https://codeup.com/uploads/247-1474999368.pdf

Parsed https://codeup.com/mailto:martab7.am@gmail.com

Parsed https://codeup.com/uploads/191-1462245926.pdf

Parsed https://codeup.com/mailto:cjsampson41@gmail.com

Parsed https://codeup.com/mailto:randimays1024@gmail.com

Parsed https://codeup.com/mailto:www.bitedge.co

Parsed https://codeup.com/uploads/198-1466177335.pdf

Parsed htt


Parsed https://codeup.com/mailto:nicholasgswan@gmail.com

Parsed https://codeup.com/mailto:po.grammer.lin@gmail.com

Parsed https://codeup.com/mailto:juanfcandia@gmail.com 

Parsed https://codeup.com/uploads/391-1535398922.pdf

Parsed https://codeup.com/mailto:danielstrolle@gmail.com

Parsed https://codeup.com/mailto:sukari.r.jackson@gmail.com

Parsed https://codeup.com/uploads/374-1512669631.pdf

Parsed https://codeup.com/mailto:anderson.d.cardoso@hotmail.com

Parsed https://codeup.com/mailto:kelleyc718@gmail.com

Parsed https://codeup.com/mailto:ssaegert@gmail.com

Parsed https://codeup.com/uploads/432-1524240951.pdf

Parsed https://codeup.com/mailto:adrian.perez.456@gmail.com

Parsed https://codeup.com/uploads/386-1512541526.pdf

Parsed https://codeup.com/mailto:chasehammock@gmail.com

Parsed https://codeup.com/mailto:stephencisler@gmail.com

Parsed https://codeup.com/uploads/411-1596726877.pdf

Parsed https://codeup.com/mailto:paul.stevenson1015@gmail.com

Parsed https://codeup.co


Parsed https://codeup.com/uploads/303-1489871501.pdf

Parsed https://codeup.com/uploads/293-1485382489.pdf

Parsed https://codeup.com/mailto:patrick.k.woolard@gmail.com

Parsed https://codeup.com/uploads/259-1477876801.pdf

Parsed https://codeup.com/mailto:jmg0343@gmail.com

Parsed https://codeup.com/mailto:geist72@gmail.com

Parsed https://codeup.com/uploads/273-1484511302.pdf

Parsed https://codeup.com/mailto:vanessamnoble@gmail.com

Parsed https://codeup.com/uploads/301-1492109895.pdf

Parsed https://codeup.com/mailto:austinmeeker@me.com

Parsed https://codeup.com/uploads/291-1485549610.pdf

Parsed https://codeup.com/mailto:nick@nickturner.tech

Parsed https://codeup.com/mailto:lloyd.h.garrison@gmail.com

Parsed https://codeup.com/uploads/282-1485404451.pdf

Parsed https://codeup.com/uploads/278-1486056412.pdf

Parsed https://codeup.com/mailto:dukecodes@gmail.com

Parsed https://codeup.com/uploads/281-1485827896.pdf

Parsed https://codeup.com/mailto:afortney86@gmail.com

Parsed htt

Parsed /checkpoint/rp/request-password-reset?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2FshareArticle%3Ftoken%26isFramed%3Dfalse%26url%3Dhttps%253A%252F%252Fsanantonioreport%2Eorg%252Fcodeup-grads-released-general-population-2%252F
Invalid URL '/signup/cold-join?session_redirect=https%3A%2F%2Fwww.linkedin.com%2FshareArticle%3Ftoken%26isFramed%3Dfalse%26url%3Dhttps%253A%252F%252Fsanantonioreport.org%252Fcodeup-creates-crowdfunding-tuition-program-2%252F': No schema supplied. Perhaps you meant http:///signup/cold-join?session_redirect=https%3A%2F%2Fwww.linkedin.com%2FshareArticle%3Ftoken%26isFramed%3Dfalse%26url%3Dhttps%253A%252F%252Fsanantonioreport.org%252Fcodeup-creates-crowdfunding-tuition-program-2%252F?
Parsed /signup/cold-join?session_redirect=https%3A%2F%2Fwww.linkedin.com%2FshareArticle%3Ftoken%26isFramed%3Dfalse%26url%3Dhttps%253A%252F%252Fsanantonioreport.org%252Fcodeup-creates-crowdfunding-tuition-program-2%252F
Invalid URL '/checkpoint/rp/request-password-reset?ses

Parsed https://codeup.com/learn-to-code-using-your-gi-bill/
Parsed https://codeup.com/page/12/?s=data&amp;et_pb_searchform_submit=et_search_proccess&amp;et_pb_include_posts=yes
Parsed https://codeup.com/jobs-bulletin/
Parsed https://codeup.com/careers-2/
Parsed https://codeup.com/page/13/?s=data&amp;et_pb_searchform_submit=et_search_proccess&amp;et_pb_include_posts=yes
Parsed https://codeup.com/women/
string index out of range
Parsed https://codeup.com/testing-the-page/
Parsed https://codeup.com/page/14/?s=data&amp;et_pb_searchform_submit=et_search_proccess&amp;et_pb_include_posts=yes
Parsed https://codeup.com/remote/
Parsed https://codeup.com/career-change/
Parsed https://codeup.com/page/15/?s=data&amp;et_pb_searchform_submit=et_search_proccess&amp;et_pb_include_posts=yes
Parsed https://codeup.com/page/16/?s=data&amp;et_pb_searchform_submit=et_search_proccess&amp;et_pb_include_posts=yes
Parsed https://codeup.com/page/17/?s=data&amp;et_pb_searchform_submit=et_search_proccess&amp;et_pb_

## Bonus Scrape all Codeup Blogs

In [5]:
df.shape

(1588, 7)